# AMADEUS CHALLENGE 

## Archivos de trabajo: bookings.csv y searches.csv


### -Ejercicio 1: Count the number of lines in Python for each file

### Resuelto con la Shell

In [ ]:
#sacamos un sample para trabajar sobre él con mayor comodidad y agilidad

In [1]:
! bzcat ./bookings.csv.bz2 | head -10000 > ./bookings.sample.csv


bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = ./bookings.csv.bz2, output file = (stdout)


- bzip2 otro compresor para archivos grandes, garantiza que dentro solo hay un archivo por lo que puedes trabajar sobre el comprimido sabiendo qué archivo estás modificando

In [ ]:
# ! bzip2 nombre_fichero para comprimir, ! bunzip2 *.bz2 para descomprimir

In [ ]:
! bzip2 -f ./bookings.sample.csv #-fpara que no de error al ejecutar varias veces

In [ ]:
# hemos pasado el sample otra vez al formato original por costumbre, no es obligatorio

In [ ]:
pwd

In [ ]:
! bzcat ./bookings.sample.csv.bz2 | wc -l

### Resuelto con Python

In [ ]:
- a)sin descomprimir
- b)using row uncompressed files

In [ ]:
import bz2

In [ ]:
fileBz2 = bz2.BZ2File('./bookings.sample.csv.bz2')

In [ ]:
type(fileBz2)

In [ ]:
k = 0
for line in fileBz2:
    k+=1
print(k) 

-Buena practica unir en una sola celda lo que comprobamos que funciona hasta ahora

In [ ]:
fileBz2 = bz2.BZ2File('./bookings.sample.csv.bz2')
k = 0
for line in fileBz2:
    k+=1
print(k) 

-importante cerrar fichero porque si estamos leyendo muchos nos consume la memoria

In [ ]:
fileBz2.close()

In [ ]:
fileBz2.closed

-Mejor usar with para que se cierre automaticamente el archivo y no nos preocupamos

In [ ]:
with bz2.BZ2File('./bookings.sample.csv.bz2') as fileBz2:
    k = 0
    for line in fileBz2:
        k+=1
    print(k) 

In [ ]:
fileBz2.closed

-Leemos y cerramos el fichero con try y except para evitar que nos de error, y cubrir el caso de que el archivo no exista

In [ ]:
try:
    with bz2.BZ2File('./bookingsj.sample.csv.bz2') as fileBz2:
        k = 0
        for line in fileBz2:
            k+=1
        print(k) 
except FileNotFoundError:
    print('File not found')
except:
    print('unexpected error')

-Esta es la forma más profesional de hacerlo

In [ ]:
try:
    with bz2.BZ2File('./bookingsj.sample.csv.bz2') as fileBz2:
        k = 0
        for line in fileBz2:
            k+=1
        print(k) 
except FileNotFoundError as message:
    print(message)
except:
    print('unexpected error')

- Codigo aún mas profesional

In [ ]:
try:
    with bz2.BZ2File('./bookings.sample.csv.bz2') as fileBz2:
        k = 0
        for k,line in enumerate(fileBz2):
                pass #no hace nada
        print(k+1) 
except FileNotFoundError as message:
    print(message)
except:
    print('unexpected error')

### Resuelto con Pandas

In [ ]:
import pandas as pd

-Sacamos los nombres de las columnas

In [ ]:
! bzcat bookings.sample.csv.bz2 | head -1 | tr ^'\n'

-Elegimos solo una columna para no leer el fichero entero

In [ ]:
df = pd.read_csv('bookings.sample.csv.bz2', sep='^', usecols = ['pax'])

-Nos saldrán 9999 líneas porque una es la cabecera

In [ ]:
df.shape

In [ ]:
len(df)

### -Ejercicio 2: Plot the monthly number of searches for flights arriving at Málaga, Madrid or Barcelona

-inicialmente yo lei el archivo con el atributo nrrows 10.000 lineas sobre el fichero original en lugar de sobre el sample, tambien se podría hacer así

In [ ]:
import pandas as pd

In [ ]:
b = pd.read_csv('bookings.sample.csv.bz2', sep='^') 

#        PLAN DE ACCION:

## 1. Nos familiarizamos con los datos

In [ ]:
b.shape

In [ ]:
b.describe() # nos devuelve solo las columnas que son numéricas

In [ ]:
b.head()

-La función sample te devuelve 5 filas aleatorias del archivo

In [ ]:
b.sample(5)

-Estos comandos (head,sample,tail) no nos sirve porque no podemos ver las 38 columnas, para eso tenemos el set_option, con None no hay limite de columnas, si ponemos numero esas son las que nos enseñan

In [ ]:
pd.set_option('display.max_columns',None)

In [ ]:
b.sample(5)

-Primero analizamos los datos. Por ejemplo tenemos 4 columnas con datetime. Act_date es la fecha de la ultima actividad, cre_date es cuando se crea la reserva.
La source es el operador que lo gestiona, el GDS (como Amadeus). Pos_iata son cosas hasheadas, que han pasado por un algoritmo y devuelve esos numeros, no se puede volver a un paso previo y ver de donde viene, las compañias lo utilizan para protegerse. Distance =0 representan los ida y vuelta, salen y vuelven al mismo sitio. Columna route, iniciales de la ruta realizada, cada ciudad son 3 letras, si hay 9 significa que ha habido una conexion

In [ ]:
b.groupby('rloc')['act_date'].count().sort_values(ascending=False)

In [ ]:
list(b.columns)

-el nombre de la columnsa tenia espacios por eso no lo reconoce

-rloc es el numero de reserva

In [ ]:
b.groupby('rloc          ')['act_date           '].count().sort_values(ascending=False)

In [ ]:
b[b['rloc          '] == 'fb72a3899ed1cd353c5830388935e7f5'].sort_values('act_date           ',ascending=True).head()

In [ ]:
b.describe(include='all')

-Con este describe podemos ver directamente lo que hemos hecho co el codigo anternior para ver que en rloc el maximo son 42

In [ ]:
b.info()

In [ ]:
b.isnull().sum() # no es normal no tener nulls en una tabla, es una tabla super limpia (primeros 10.000)

## 2. Seleccionamos las columnas de interes

In [ ]:
b = pd.read_csv('bookings.sample.csv.bz2', sep='^',  usecols = ['arr_port', 'pax', 'year'])
# podemos hacerlo sobre el fichero grande y seleccionar nrows=9999, es lo mismo

In [ ]:
b.head()

- Qué hacemos con los NULL

In [ ]:
b = b.dropna()

-PLAN DE ACCION:
        - filtrar 2013
        - groupby sobre arr_port
        - sumar pax
        - ordenar
        - top 10

In [ ]:
b = b[b['year']==2013]



In [ ]:
len(b)

In [ ]:
del b['year'] # year ya no te sirve para nada

In [ ]:
b = b.dropna()
b = b[b['year']==2013]
del b['year'] # year ya no te sirve para nada
top = b.groupby('arr_port').sum().sort_values(by='pax', ascending= False)
top[:10]

In [ ]:
top.reset_index().head()

-hay que unir todo lo anterior en una celda para sacar le codigo limpio, desde pd.set.option hasta el top

- Hemos sacado Top 10 con 10.000 datos, ahora hay que extrapolarlo al archivo entero con los chunks
### Chunks

In [ ]:
bi = pd.read_csv('bookings.csv.bz2', sep='^',  usecols = ['arr_port', 'pax', 'year'], nrows = 9999, iterator=True )
#el iterator es la clave para que vaya recorriendo los chunks y vaya avanzando por el archivo,no definimos que numero de elementos

In [ ]:
type(bi)

In [ ]:
b = bi.get_chunk(6000) # si hacemos otra vez get empezamos a partir de 6000

In [ ]:
type(b) # esto es lo que leiamos antes con el read_csv, un DF

In [ ]:
b.head()

In [ ]:
b1 = bi.get_chunk(3000)
b1.head(3)

In [ ]:
b2 = bi.get_chunk(300000000)
b2.shape

- no devuelve error porque hemos definido con nrows que cuando lleguemos a 9999 para, es decir, no tenemos que iterar con el numero exacto de elementos del fichero, cuando llegue al final del fichero para la iteración

In [ ]:
bi = pd.read_csv('bookings.csv.bz2', sep='^',  usecols = ['arr_port', 'pax', 'year'], nrows = 9999, chunksize=3000)
# el iterator=True se puede quitar, funciona igualmente
all_chunks=pd.DataFrame() # aqui van a ir todos los chunks
for i,b in enumerate(bi):
    print(i)
    print(len(b))
    #ahora ponemos el codigo de antes, que vamos a repetir para todos los chunks
    b = b.dropna()
    b = b[b['year']==2013]
    del b['year'] # year ya no te sirve para nada
    top = b.groupby('arr_port').sum().sort_values(by='pax', ascending= False)
    top.reset_index(inplace=True)
    all_chunks = all_chunks.append(top)
    
    
all_result = all_chunks.groupby('arr_port').sum().sort_values(by='pax', ascending= False).reset_index()


In [ ]:
all_chunks.shape

In [ ]:
all_result.head(10)

In [ ]:
-Ya lo hemos probado y funciona el codigo, ahora lo hacemos sobre el archivo entero

In [ ]:
%%time
bi = pd.read_csv('bookings.csv.bz2', sep='^',  usecols = ['arr_port', 'pax', 'year'],  chunksize=1000000)
# el iterator=True se puede quitar, funciona igualmente
all_chunks=pd.DataFrame() # aqui van a ir todos los chunks
for i,b in enumerate(bi):
    print(i)
    print(len(b))
    #ahora ponemos el codigo de antes, que vamos a repetir para todos los chunks
    b = b.dropna()
    b = b[b['year']==2013]
    del b['year'] # year ya no te sirve para nada
    top = b.groupby('arr_port').sum().sort_values(by='pax', ascending= False)
    top.reset_index(inplace=True)
    all_chunks = all_chunks.append(top)
    
    
all_result = all_chunks.groupby('arr_port').sum().sort_values(by='pax', ascending= False).reset_index()

In [ ]:
all_result.head(10)

## Bonus point del ejercicio 2, GeoBase


In [ ]:
! pip install Neobase

In [ ]:
import neobase as nb

In [ ]:
from neobase import NeoBase

In [ ]:
geoDict = NeoBase()

In [ ]:
type(geoDict)

In [ ]:
geoDict.get('LHR') # nombre aeropuerto

In [ ]:
type(geoDict.get('LHR') )

In [ ]:
geoDict.get('LHR')['name']

In [ ]:
all_result.head()

In [ ]:
all_result['arr_port'][0]

In [ ]:
- Esto hay que arreglarlo, columna con espacios

In [ ]:
all_result['arr_port'][0].strip() #.strip quita los espacios en blanco a ambos lados

- Si tenemos espacios en blanco en el medio, funciones split y join

In [ ]:
'skdfsljf d    h    dkfdkfjdfjdfk'.split()

In [ ]:
' '.join('skdfsljf d    h    dkfdkfjdfjdfk'.split())

In [ ]:
all_result['arr_port'] =all_result['arr_port'].str.strip()

In [ ]:
all_result['AirportName'] = all_result['arr_port'].map(lambda x:geoDict.get(x)['name']) 
# este error quiere decir que el eropuerto CPQ no esta en el otro sitio

In [ ]:
all_result[all_result['arr_port']=='CPQ'] # da error porque CPQ no tiene name, comprobar!!!!!!

In [ ]:
all_result = all_result[all_result['arr_port']!='CPQ']

-Ahora si aplicamos el mismo codigo

In [ ]:
all_result['AirportName'] = all_result['arr_port'].map(lambda x:geoDict.get(x)['name']) 


In [ ]:
all_result.head()

In [ ]:
#ver qué es astype

In [ ]:
all_result.shape

In [ ]:
all_result.to_csv('top_airports.csv', sep='^',index=False)

In [ ]:
ls

In [ ]:
! head -2 top_airports.csv

### - Ejercicio 5: Write a Web Service: Wrap the output of the second exercise in a web service that returns the data in JSON format (instead of printing to the standard output). The web service should accept a parameter n>0. For the top 10 airports, n is 10. For the X top airports, n is X 

In [ ]:
#el webservice es un api, lo que esta trabajando en la shell cuando abrimos notebook es un WS, el local host de arriba

In [1]:
from flask import Flask

In [ ]:
app =Flask('My first web service')
@app.route('/hello', methods=['GET'])
#GET The browser tells the server to just get the infomation stored
def get_hello():
             return 'Hello DS from the service!'


In [ ]:
app.run()
#running os http://127.0.0.1:5000/

In [ ]:
#si copiamos http://127.0.0.1:5000/hello en otra ventana te devuelve el mensaje

In [2]:
app =Flask('My first web service')
@app.route('/hello', methods=['GET'])
#GET The browser tells the server to just get the infomation stored
def get_hello():
             return 'Hello DS from the service!'
@app.route('/ret_number/<int:n>',methods=['GET'])
def get_number(n):
    return 'i got %d'%n

In [ ]:
app.run()

 * Serving Flask app "My first web service" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Jan/2020 13:42:31] "GET /ret_number/10 HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2020 13:43:23] "GET /ret_number/10 HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2020 13:43:24] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
#pones en una ventana http://127.0.0.1:5000/ret_number/10 y tienes conexion con ese host

In [ ]:
# el Viernes vamos a aplicar esto al archivo del top ten , intentar